This code can be used to create sqllite db file from the preprocessed csv data files.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# FILE_NAME = 'lastfm.db'
FILE_NAME = 'lastfm.db'

In [3]:
listenings = pd.read_csv('preprocessed_data/listenings_with_years_and_users.csv',parse_dates=['timestamp','album_date'])
# no timezones needed
listenings['timestamp'] = listenings['timestamp'].dt.tz_localize(None)
listenings['album_date'] = listenings['album_date'].dt.tz_localize(None)

In [4]:
europeList = ['United Kingdom', 'Finland', 'Germany', 'Sweden', 'Canada', 'Switzerland',
    'Norway', 'Poland', 'Netherlands', 'Belgium', 'Italy', 'Romania',
       'Russian Federation', 'Spain',
       'Ireland','Czech Republic','France', 'China',
       'Austria', 'Slovakia', 'Malta',
       'Latvia', 'Estonia', 'Lithuania',
      'Iceland', 'Bulgaria',
      'Portugal', 'Croatia',  'Serbia', 'Greece', 'Hungary',
       'Bosnia and Herzegovina', 'Slovenia',
       'Belarus',  'Macedonia']
# we rename the European countries so that we have a bigger geographical group which we can compare to the United States
listenings['country'] = listenings['country'].apply(lambda x: 'Europe' if x in europeList else x)

In [5]:
listenings.country.unique()

array(['Japan', 'Argentina', nan, 'Europe', 'United States', 'Armenia',
       'Mexico', 'Turkey', 'Brazil', 'New Zealand', 'Morocco',
       'Australia', 'Chile', 'Colombia', 'Algeria', 'Israel', 'Thailand',
       'United States Minor Outlying Islands', 'Venezuela', 'Singapore',
       'Trinidad and Tobago', 'Antarctica',
       'British Indian Ocean Territory',
       "Korea, Democratic People's Republic of", 'Peru', 'India',
       'Northern Mariana Islands',
       'Congo, the Democratic Republic of the', 'Tunisia', 'Zimbabwe',
       'Nicaragua', "Cote D'Ivoire", 'Netherlands Antilles'], dtype=object)

In [6]:
# listenings = listenings[listenings['country']=='Europe']

In [7]:
# we only take a look at songs released in the year 2005 and later (when the dataset begins)
listenings = listenings[listenings['album_date']>=min(listenings['timestamp'])]
listenings = listenings[listenings['album_date']<=max(listenings['timestamp'])]
# we create a unique identifier from artist and track name
listenings['art_tra_name'] = listenings['artname'] + '@' + listenings['traname']

In [8]:
listenings.head()

,userid,timestamp,artname,traname,album_id,album_date,gender,country,art_tra_name
41,user_000001,2008-12-03 01:28:24,Underworld,"Boy, Boy, Boy (Switch Remix)",8a6dd62e-3d3c-4812-a92a-46235400354f,2007-12-12,m,Japan,"Underworld@Boy, Boy, Boy (Switch Remix)"
42,user_000001,2008-07-25 01:32:09,Underworld,"Boy, Boy, Boy (Switch Remix)",8a6dd62e-3d3c-4812-a92a-46235400354f,2007-12-12,m,Japan,"Underworld@Boy, Boy, Boy (Switch Remix)"
43,user_000001,2008-07-15 13:44:52,Underworld,"Boy, Boy, Boy (Switch Remix)",8a6dd62e-3d3c-4812-a92a-46235400354f,2007-12-12,m,Japan,"Underworld@Boy, Boy, Boy (Switch Remix)"
44,user_000001,2008-07-14 16:42:49,Underworld,"Boy, Boy, Boy (Switch Remix)",8a6dd62e-3d3c-4812-a92a-46235400354f,2007-12-12,m,Japan,"Underworld@Boy, Boy, Boy (Switch Remix)"
45,user_000001,2008-07-14 15:08:59,Underworld,"Boy, Boy, Boy (Switch Remix)",8a6dd62e-3d3c-4812-a92a-46235400354f,2007-12-12,m,Japan,"Underworld@Boy, Boy, Boy (Switch Remix)"


In [9]:
# get unique songs with release years
nodes = listenings[['art_tra_name','album_date']].drop_duplicates().copy()
nodes.rename({'art_tra_name':'name','album_date':'realtime'},axis=1,inplace=True)
# we only have years which we use as dates
# nodes['realtime'] = pd.to_datetime(nodes['realtime'], format='%Y-%M')
nodes.sort_values('realtime',inplace=True)
nodes.head()

,name,realtime
2324347,Chris Rock@Tip Your Hat To Whitey (Mars),2005-02-15
3080183,Neuroticfish@Short Commercial Break,2005-02-15
3080180,"Neuroticfish@Die Bombe, Die Nicht Tickt",2005-02-15
2324344,Chris Rock@Ballmart,2005-02-15
49716,Neuroticfish@They Are Coming To Take Me Away,2005-02-15


In [10]:
# primary key
nodes['id_nb'] = range(0, len(nodes))

In [11]:
# we create the intrinsictime column
realtimeDistinct = sorted(list(set(nodes['realtime'])))
realtimeDict = {}
for i,realtimeValue in enumerate(realtimeDistinct):
    realtimeDict[realtimeValue] = i
nodes['intrinsictime'] = nodes['realtime'].apply(lambda x: realtimeDict[x])

# monthly bins
realtimeBinDict = {}
yearMonthDistinct = sorted(list(set([str(x.year) + str(x.month).zfill(2) for x in realtimeDistinct])))

for i,yearMonthValue in enumerate(yearMonthDistinct):
    realtimeBinDict[yearMonthValue] = i
    
nodes['year_month'] = nodes['realtime'].apply(lambda x: str(x.year)) + nodes['realtime'].apply(lambda x: str(x.month).zfill(2))
nodes['timebin'] = nodes['year_month'].apply(lambda x: realtimeBinDict[x])
nodes.drop('year_month',axis=1,inplace=True)

#nodes = nodes[nodes['timebin']!=5]
nodes

,name,realtime,id_nb,intrinsictime,timebin
2324347,Chris Rock@Tip Your Hat To Whitey (Mars),2005-02-15,0,0,0
3080183,Neuroticfish@Short Commercial Break,2005-02-15,1,0,0
3080180,"Neuroticfish@Die Bombe, Die Nicht Tickt",2005-02-15,2,0,0
2324344,Chris Rock@Ballmart,2005-02-15,3,0,0
49716,Neuroticfish@They Are Coming To Take Me Away,2005-02-15,4,0,0
...,...,...,...,...,...
5446730,Doyle W. Donehoo@Ancient Rites (Eldar Theme),2009-04-29,140858,1501,50
5446729,Doyle W. Donehoo@The Green Horde Rises,2009-04-29,140859,1501,50
5446728,Doyle W. Donehoo@Khaine'S Wrath,2009-04-29,140860,1501,50
5446736,Doyle W. Donehoo@Angels Of Death (Space Marine...,2009-04-29,140861,1501,50


In [12]:
nameToIdDict = {}
for name,id_nb in zip(nodes['name'],nodes['id_nb']):
    nameToIdDict[name] = id_nb

stream = listenings[['timestamp','art_tra_name']].copy()
# convert the song names to the id used in other table
stream['node'] = stream['art_tra_name'].apply(lambda x: nameToIdDict[x])
# stream['timestamp'] = pd.to_datetime(stream['timestamp'], format='%Y-%m-%d %H:%M:%S%z')
stream['node_origin'] = None
stream.rename({'timestamp':'realtime'},axis=1,inplace=True)

stream.drop('art_tra_name',axis=1,inplace=True)
stream.head()

,realtime,node,node_origin
41,2008-12-03 01:28:24,110358,None
42,2008-07-25 01:32:09,110358,None
43,2008-07-15 13:44:52,110358,None
44,2008-07-14 16:42:49,110358,None
45,2008-07-14 15:08:59,110358,None


In [13]:
# add intrinsictime collumn
realtimeDistinct = sorted(list(set(stream['realtime'])))
realtimeDict = {}
for i,realtimeValue in enumerate(realtimeDistinct):
    realtimeDict[realtimeValue] = i
stream['intrinsictime'] = stream['realtime'].apply(lambda x: realtimeDict[x])

In [14]:
# monthly bins
realtimeBinDict = {}

stream['year_month'] = stream['realtime'].apply(lambda x: str(x.year)) + stream['realtime'].apply(lambda x: str(x.month).zfill(2))
yearMonthDistinct = sorted(list(set(stream['year_month'])))

for i,yearMonthValue in enumerate(yearMonthDistinct):
    realtimeBinDict[yearMonthValue] = i
    
stream['timebin'] = stream['year_month'].apply(lambda x: realtimeBinDict[x])
stream.drop('year_month',axis=1,inplace=True)


# binDistinct = sorted(list(set(stream['realtime'].apply(lambda x: x.year))))
# binDict = {}
# for i,binValue in enumerate(binDistinct):
#     binDict[binValue] = i
# stream['timebin'] = stream['realtime'].apply(lambda x: x.year).apply(lambda x: binDict[x])
stream.sort_values('realtime')

,realtime,node,node_origin,intrinsictime,timebin
2879977,2005-02-24 15:04:15,956,None,0,0
2879976,2005-02-24 15:34:28,956,None,1,0
2879975,2005-02-24 15:45:26,956,None,2,0
6014152,2005-02-24 16:44:57,956,None,3,0
2879974,2005-02-25 03:34:16,956,None,4,0
...,...,...,...,...,...
3609623,2009-04-30 23:52:36,100018,None,1613296,50
2043493,2009-04-30 23:53:50,2449,None,1613297,50
3618306,2009-04-30 23:56:07,104848,None,1613298,50
1957994,2009-04-30 23:56:24,118141,None,1613299,50


In [15]:
import sqlite3 as sql

conn = sql.connect(f'preprocessed_data/{FILE_NAME}')
nodes.to_sql('nodes',conn)
stream.to_sql('stream', conn)

In [16]:
conn.close()